# Output models

Class models for different types of output files

In [84]:
from pydantic import BaseModel, ValidationError, validator, Field
from pydantic.generics import GenericModel
%run marxan_utils.ipynb

In [104]:
MARXAN_FOLDER = '/home/jovyan/work/datasets/raw/Marxan_BLM/BLM_0.001'
InputFile = DatFile(f'{MARXAN_FOLDER}/input.dat')
InputFile.read()
userInputFile = inputDatFile.from_dat(InputFile.data)
filename = f'{MARXAN_FOLDER}/{userInputFile.OUTPUTDIR}/{userInputFile.SCENNAME}_r00001.csv'

In [107]:
class OutputRun(BaseModel): 
    """
    Class of files _r00001.csv and _best.csv
    """
# General Parameters
    PUID: int = Field(title='Planning Unit id', 
                         description='List of planning units')
    SOLUTION:  int = Field(title='Solution', 
                           description='Planning unit included in this solution if 1, not included if 0',
                           ge =0, le =1)

In [106]:
CreateListModelFromFile(filename, OutputRun)[0:2]

[OutputRun(PUID=1, SOLUTION=1), OutputRun(PUID=2, SOLUTION=0)]

In [111]:
class OutputMV(object):
    """
    Class of files mv00001.csv
    """
# General Parameters
    Conservation_Feature: int = Field(title = 'Conservation feature id',
                                      description ='The unique ID number of the conservation feature')
    Feature_Name: str = Field(title = 'Feature Name',
                               description = 'The optional alphabetic name of the conservation feature.\
                               If no name has been specified then nothing will appear in this column.')
    Target: float = Field(title= 'target',
                           description = 'The target level of representation (if any) for that conservation feature')
    Amount_Held: float = Field(title = 'Amount held',
                                description = 'The amount of that conservation feature captured in the reserve system')
    Occurrence_Target: float = Field(title = 'Occurrence target',
                                     description ='The target number of occurrences in the reserve system for that conservation feature')
    Occurrences_Held: float = Field(title= 'Ocurrence s held',
                                    description= 'The number of occurrences of the conservation feature captured in the solution.\
                                    Again, only occurrences in valid clumps are included')
    Separation_Target: float = Field (title= 'Separation target',
                                     description = 'The number of mutually and adequately separated occurrences of that\
                                     conservation feature required in the reserve system')
    Separation_Achieved: float = Field(title = 'Separation Achieved',
                                      description= 'The number reported here will be the lowest of either: the number of \
                                      separate occurrences that are actually achieved in the reserve system ; or the target \
                                      number of separate occurrences. The separation count (see Appendix B-1.3.1) never exceeds \
                                      the separation target for that feature. This is a convention which speeds up the execution \
                                      of the software but it means that no information is given about how far this target is exceeded.' )
    Target_Met: str = Field(title= 'Target met',
                           description = 'An alphabetic variable that returns ‘yes’ if all the targets set for that feature are met,\
                           otherwise it returns ‘no’')
    MPM: float = Field (title= 'Minimum Proportion Met',
                       description= 'Propotion of target met, min in 0 max i 1')

In [ ]:
class OutputSolMatrix(object):
    """
    Class of files solutionsmatrix.csv
    """

In [ ]:
class OutputSsoln(object):
    """
    Class of files solutionsmatrix.csv
    """

In [ ]:
class OutputSum(object):
    """
    Class of files solutionsmatrix.csv
    """

In [ ]:
class OutputSen(object):
    """
    Class of files solutionsmatrix.csv
    """

In [ ]:
class OutputLog(object):
    """
    Class of files solutionsmatrix.csv
    """

## A second heading

and some more text